In [1]:
import pandas as pd
import numpy as np

In [ ]:
outs = pd.read_csv('Supp_table_all_variants.SVs_included.csv')
outs = outs.set_index('gene', drop=False)

rel = pd.read_csv('../data/family_summaries.tsv', sep='\t')
rel = rel.set_index('subject', drop=False)
rel = rel.drop('SG011')

pheno = pd.read_csv('../data/pheno_final.tsv', sep='\t')
pheno = pheno.drop_duplicates('subject')
pheno = pheno.set_index('subject', drop=False)

mapp = pd.read_csv('../data/gene_names_mapping_new.tsv', sep='\t')
mapp = mapp.set_index('ensembl', drop=False)

# format network df

In [ ]:
net = pd.read_csv('all_gene_network_simple.tsv', sep='\t')

net.columns = ['startg', 'endg', 'distance',
       'num_connector_genes']

net = net.set_index(['startg', 'endg'], drop=False)

net = net.rename_axis((None, None))

## create df to hold output statistics

In [ ]:
df = pd.DataFrame(index=pheno.subject)
df['avg_shortest_distance'] = 0.0

# outlier genes to wg variants

In [ ]:
snv = pd.read_csv('../network_analysis/16p_all_snvs.csv', sep=',')

def str2float(s, default = 0):
    if s == '.':
        return default
    else:
        return float(s)

    
snv.CADD = snv.CADD.apply(lambda s: str2float(s))

lof = snv.Mut_type == 'lof'
cadd_10 = snv.CADD > 10
splice = snv.Mut_type == 'splice'

snv = snv[lof | cadd_10 | splice].copy()

In [6]:
cnv = pd.read_csv('../../cnv_parse/output/merged/16p_all_rare_cnvs.tsv', sep='\t')
cnv = cnv[cnv.gene_covered == 'encapsulated']

In [22]:
de = pd.DataFrame()


for sub in rel.subject:
    for ltype in ['de_novo', 'inherited_from_carrier', 'inherited_from_non_carrier']:
        filename = '../family_based_analysis3/output/three_lists/{}.{}.tsv'.format(sub, ltype)
#         print(filename)
        app = pd.read_csv(filename, sep='\t')
        app = app[['ensembl']]
        app['subject'] = sub
        app['ltype'] = ltype
        de = de.append(app)

de['gene'] = mapp.loc[de.ensembl.to_list()]['Description'].to_list()

In [21]:
for sub in rel.subject:
    print(sub)
    de_genes = de[(de.subject == sub)].gene.to_list()
    
    snv_genes = snv[snv['Sample'] == sub]
    snv_genes = list(snv_genes['Gene.refGene'].unique())
    cnv_genes = cnv[cnv.subject == sub].gene.to_list()
    variant_genes = list(set(snv_genes + cnv_genes))



    subnet = net[net.endg.isin(variant_genes)]

    subnet = subnet[subnet.startg.isin(de_genes)]

    subnet = subnet[subnet.startg != subnet.endg]

    df.at[sub, 'avg_shortest_distance'] = subnet.num_connector_genes.mean()

SG001
SG006
SG021
SG022
SG023
SG155
SG069
SG026
SG041
SG037
SG044
SG040
SG038
SG042
SG150
SG149
SG148


In [23]:
df = df[df.avg_shortest_distance != 0]

In [25]:
df.to_csv('avg_dist_famde.tsv', sep='\t')

In [24]:
df

,avg_shortest_distance
subject,
SG001,3.412339
SG006,3.471127
SG021,3.387186
SG022,3.384053
SG023,3.370561
SG026,3.429454
SG069,3.367798
SG155,3.355617
SG037,3.428624
